# BOTPT Data detide for Central Caldera Station E; Chadwick Method

We detide seafloor pressure measurements taken at The Axial Seamount's Central Caldera (RS03CCAL)  

# Abstract
#### The Ocean Observatories Initiative Cabled Array (OOI-CA) provides the opportunity to use new approaches to observe geological  processes  at Axial Seamount, an active submarine volcano on the Juan de Fuca spreading ridge located ~300 miles off the coast of Oregon. The OOI-CA provides a suite of interdisciplinary real-time datasets from seismic and geodetic networks and a variety of instruments in two hydrothermal fields with which we can examine system-level processes governing the volcano-marine environment. The geophysical networks comprise 4 Pressure/Tilt (BOTPT) instruments that detect the rise, fall, and tilt of the seafloor as it responds to the movement of magma within the crust and 7 seismometers to monitor earthquakes. In order to better understand the patterns of magma storage and delivery beneath the Axial summit caldera and provide constraints for the development of improved predictive models, we are examining short-term variations in the rates of deformation and seismicity and how well they co-vary during the inter-eruption time period. We will also investigate apparent “pauses” in the magma supply rate when the rate of seismicity and deformation have both dropped. On the time-scale of years, seafloor deformation measurements show a relatively steady rate of inflation of the seafloor between eruptions that can be interpreted as reflecting the rate that magma is supplied to the sub-caldera magma reservoir (Nooner and Chadwick 2016). However, on the time-scale of weeks to months, the rate is more variable. Similarly, the rate of earthquakes detected by the OOI seismometer network also varies with time. It was very low immediately after the 2015 eruption, but it has increased markedly over the past year, and based on the observations of prior eruptive cycles, it is expected to increase by several orders of magnitude before the next eruption.

In [ ]:
%reset -f

In [ ]:
# Load map of Axial caldera showing locations of BOTPT instruments (red circles)
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-caldera-ed-sm.png")

In [ ]:
#Zoomed in map of Axial caldera
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-zoom-ed-sm.png")

### API Information Setup

In [ ]:
USERNAME = 'OOIAPI-J97520T1AYPUNI'
TOKEN =  'TEMP-TOKEN-YD01XSNDIO57MP'

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc
import warnings
warnings.filterwarnings('ignore')

#### For mad help with Pandas https://chrisalbon.com
#### For all sorts of great repos https://github.com/rabernat/research_computing.git

#### Read Tide Data E

In [ ]:
tide_file = '/home/jovyan/data/botpt/tidepredictions/pred_E.txt'
df_tides = pd.read_csv(tide_file, delim_whitespace=True, dtype = object)
df_tides['datetime']=df_tides['year'] + '-' + df_tides['month'] + '-' + df_tides['day'] + \
            'T' + df_tides['hour'] + ':' + df_tides['minute'] + ':' + df_tides['second']
#df_tides_E = pd.DataFrame(tides_E)
df_tides.index=pd.to_datetime(df_tides['datetime'].values)
#df_tides_E.index = df_tides_E['parsed_time']
del df_tides['year']
del df_tides['month']
del df_tides['day']
del df_tides['hour']
del df_tides['minute']
del df_tides['second']
del df_tides['datetime']
df_tides['height'] = df_tides['height'].astype(float)
df_tides= df_tides.sort_index()
#df_tides_E['height'] = df_tides_E['height'].to_pandas().resample('T').mean()

# columnsTitles=["datetime","height"]
# df_tides_E=df_tides_E.reindex(columns=columnsTitles)
df_tides.head()

In [ ]:
#list comprehention
epoch= [i.timestamp() for i in df_tides.index.to_pydatetime()]

In [ ]:
df_tides['epoch'] = epoch
df_tides.head()

#### Read Downsampled BOTPT Data

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_E11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botpt = pd.DataFrame(botpt_data)
df_botpt['bottom_pressure'] = df_botpt['bottom_pressure'].astype(float)
df_botpt['depth']=df_botpt['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botpt.index.to_pydatetime()]
df_botpt['epoch'] = epoch
df_botpt= df_botpt.sort_index()
df_botpt.head()

In [ ]:
df_botpt.tail()

#### Combine BOTPT with Tide Predictions

In [ ]:
df_subsetTides = df_tides.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']
df_subsetBOTPT = df_botpt.loc['2015-01-1 00:00:00':'2018-10-31 00:00:00']

In [ ]:
df_subsetBOTPT['tides'] = df_subsetTides.height
df_subsetBOTPT.head()

In [ ]:
df_subsetBOTPT['dtide']=  df_subsetBOTPT['depth'] - df_subsetBOTPT['tides']
df_subsetBOTPT.head()

In [ ]:
#time = list(df_botpt.index.values)
height = list(df_subsetBOTPT['dtide'].values)

time_int = []
time = list(pd.to_datetime(df_subsetBOTPT.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.hexbin(time_int, height, vmin=0, vmax=30, gridsize=(1500,100), mincnt=1, cmap='Greens', linewidths=0)
fig.colorbar(hb1, pad = 0.01)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2015, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()
plt.savefig('Figures/StationE_2015thruPresent.png')

In [ ]:
df_subsetBOTPT['date']=pd.DatetimeIndex(df_subsetBOTPT.index).date
df_subsetBOTPT.head()

#### Use Groupby to create one day mean measurements

In [ ]:
df_botptMean=df_subsetBOTPT.groupby('date').mean()
df_botptMean.head()

#### Create time and height vectors for plotting 

In [ ]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
height = df_botptMean['dtide'].tolist()
time_int = []
time = list(pd.to_datetime(df_botptMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

#### Plot One Day Measurements. 

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 7)
hb1 = ax.plot(time_int, height)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 15, 0, 0))
ax.set_ylim(1501.25,1502.2)
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()
plt.savefig('Figures/StationF_2017_2018_1dayAve.png')

In [ ]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
# smooth data using rolling window that chops off 95th percentile 
TwelveWeek = list(movingaverage(df_botptMean['dtide'],28))

In [ ]:
len(time_int)

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 7)
hb1 = ax.plot(time_int, TwelveWeek,linewidth=5)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 15, 0, 0))
ax.set_ylim(1501.6,1502.2)
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.gca().invert_yaxis()
plt.show()
plt.savefig('Figures/StationF_2017_2018_1dayAve.png')

#### Import Seismic Data 

In [ ]:
seismic_file = '/home/jovyan/data/hypo71_2018.dat'
df_seismic_data = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic_data['datetime'] = df_seismic_data['yyyymmdd'] + 'T' + \
            df_seismic_data['HHMM'].str.slice(start=0, stop=2) + ':' + \
            df_seismic_data['HHMM'].str.slice(start=2) 
df_seismic_data.index = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data['datetime'] = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data = df_seismic_data.loc['2017-01-1 00:00:00':'2018-10-31 00:00:00']
del df_seismic_data['yyyymmdd']
del df_seismic_data['HHMM']
del df_seismic_data['SSS.SS']
del df_seismic_data['MW']
del df_seismic_data['NWR']
del df_seismic_data['GAP']
del df_seismic_data['DMIN']
del df_seismic_data['ERH']
del df_seismic_data['ERZ']
del df_seismic_data['ID']

In [ ]:
df_seismic_data.datetime.astype(np.int64).values/1e64
df_seismic_data['date'] =pd.DatetimeIndex(df_seismic_data.datetime).date
#df_seismic_data.head()

#### Create dataframe with Earthquake frequency (count per day)
##### Note: Days with zero earthquakes are not represented in this timeseries. 

In [ ]:
df_eqMean=df_seismic_data.groupby('date').count()
del df_eqMean['Lat(D']
del df_eqMean['M)']
del df_eqMean['Lon(D']
del df_eqMean['M).1']
del df_eqMean['Depth']
del df_eqMean['RMS']
del df_eqMean['PMom']
del df_eqMean['SMom']
df_eqMean['count']= df_eqMean.datetime
del df_eqMean['datetime']
df_eqMean.columns.name = df_eqMean.index.name
df_eqMean.index.name = None
#df_eqMean.head()

#### Resample. Add days to our earthquake data where there were zero earthquakes. 

In [ ]:
idx = pd.date_range('2017-01-1 00:00:00', '2018-10-31 00:00:00')

s = df_eqMean

s.index = pd.DatetimeIndex(s.index)

s = s.reindex(idx, fill_value=0)

#### Create time and count vectors earthquake frequency vectors for plotting 

In [ ]:
count = df_eqMean['count'].tolist()
time_eq = []
time = list(pd.to_datetime(df_eqMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_eq.append(dates.date2num(i))

In [ ]:
# smooth data using rolling window that chops off 95th percentile 
count_av = list(movingaverage(df_eqMean['count'],21))

#### Comparison of Earthquake Frequency and Inflation Rate 

In [ ]:
plt.close()
fig4, (ax1,ax2) = plt.subplots(2,1)
fig4.set_size_inches(28, 14)
hb1 = ax1.plot(time_int, TwelveWeek,linewidth=5)
ax1.yaxis.grid(True)
ax1.xaxis.grid(True)
ax1.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 15, 0, 0))
ax1.set_ylim(1501.6,1502.2)
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax1.xaxis.set_major_locator(months)
ax1.xaxis.set_major_formatter(monthsFmt)
ax1.xaxis.set_minor_locator(years)
ax1.xaxis.set_minor_formatter(yearsFmt)
ax1.set_title('Caldera Inflation', fontsize=18, fontweight = 'bold')
ax1.invert_yaxis()

hb1 = ax2.plot(time_eq, count)
hb2 = ax2.plot(time_eq, count_av, linewidth=5)
ax2.yaxis.grid(True)
ax2.xaxis.grid(True)
ax2.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 15, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax2.xaxis.set_major_locator(months)
ax2.xaxis.set_major_formatter(monthsFmt)
ax2.xaxis.set_minor_locator(years)
ax2.xaxis.set_minor_formatter(yearsFmt)
ax2.set_title('Daily Seismicity', fontsize=18, fontweight = 'bold')

plt.tight_layout()
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
plt.suptitle('Comparison Between Inflation and Seismicity', fontsize=32, color= 'blue', fontweight = 'bold')
plt.subplots_adjust(top=0.90)
#plt.gca().invert_yaxis()
plt.show()